In [4]:
from __future__ import print_function
import numpy as np
import sys
import os
import numpy as np
from keras.models import Model,Sequential,model_from_json
from keras.layers import Input,BatchNormalization,Lambda,Layer,Dense,Dropout, Activation, Flatten,Embedding, LSTM, CuDNNGRU
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint
from sklearn.model_selection import train_test_split
import h5py

Using TensorFlow backend.


ImportError: Could not find 'cudart64_90.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Download and install CUDA 9.0 from this URL: https://developer.nvidia.com/cuda-toolkit

In [2]:
# Define path
pth = 'F:/DTU/Stanford/Data/_features/SHHS/'
file = '205802.h5'
ins = np.array([])
armodelMaxShape = [0,0]
spectralMaxShape = [0,0]
temporalMaxShape = [0,0]
if os.path.isdir(pth):
    files = [f.name for f in os.scandir(pth)]   
    for file in files:
        filename = pth + file
        h5file = h5py.File(filename, 'r')
        #Get the HDF5 group
        #demographics = h5file['demographics'].value.transpose()
        armodel = h5file['armodel'].shape[::-1]
        spectral = h5file['spectral'].shape[::-1]
        temporal = h5file['temporal'].shape[::-1]
        if armodel[0] > armodelMaxShape[0]:
            armodelMaxShape = armodel
        if spectral[0] > spectralMaxShape[0]:
            spectralMaxShape = spectral
        if temporal[0] > temporalMaxShape[0]:
            temporalMaxShape = temporal

In [3]:
print(armodelMaxShape)
print(spectralMaxShape)
print(temporalMaxShape)

(6414, 5)
(6414, 8)
(6414, 13)


In [6]:
# Define the model
model = Sequential()
model.add(CuDNNGRU(1, input_shape=(armodelMaxShape)))
model.add(Dense(1,activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

NotFoundError: Op type not registered 'CudnnRNN' in binary running on JONATHANS. Make sure the Op and Kernel are registered in the binary running in this process.

In [ ]:
batchSize = 512

X = np.zeros((batchSize,armodelMaxShape[0],armodelMaxShape[1]))
Y = np.empty((batchSize,1))

files = [f.name for f in os.scandir(pth)]   
for i, file in enumerate(files):
    filename = pth + file
    h5file = h5py.File(filename, 'r')

    y = h5file['demographics'].value[-1]
    x = h5file['armodel'].value.transpose()
    
    Y[i,:] = y
    X[i,:x.shape[0],:] = x 
    
    if i == batchSize - 1:
        break
        
print(Y.shape)
print(X.shape)
model.fit(X, Y, epochs=32, batch_size=8, verbose=2)
        


(512, 1)
(512, 6414, 5)
Epoch 1/32


In [198]:
print(X.shape, Y.shape)

(6, 6414, 5) (6,)


In [ ]:
#def main():
    ## Logistic regression (sckitlearn) first
    #logisticregression()
    ## Random forest (sckitlearn) second (and feature selection)
    #randomforest()
    #lightgbm()
    ## Deep learning
    #feedforwardnetwork()
    fivesecfeedforwardnetwork()

In [ ]:
   
def logisticregression():
    clear = lambda: os.system('cls')
    clear() # remove warnings
    ################################ Load data ############################
    X = np.loadtxt("../../../MATLAB/XSHHS.csv", delimiter=",")
    Y = np.loadtxt("../../../MATLAB/YSHHS.csv", delimiter=",")
    predicted = cross_validation.cross_val_predict(LogisticRegression(), X, Y, cv=10)
    print(metrics.accuracy_score(Y, predicted))
    print(metrics.classification_report(Y, predicted))

In [ ]:
def randomforest():
    clear = lambda: os.system('cls')
    clear() # remove warnings
    ################################ Load data ############################
    X = np.loadtxt("../../../MATLAB/XSHHS.csv", delimiter=",")
    Y = np.loadtxt("../../../MATLAB/YSHHS.csv", delimiter=",")
    text_file = open("../../../MATLAB/featurenames.csv", "r")
    featurenames = text_file.read().split(',')

    #accs = []
    #samples = []
    #sizes = np.arange(0.8,0.95,0.05)
    #for s in sizes:
    sss = StratifiedShuffleSplit(n_splits=10, train_size=0.9)
    acc = []
    for idx_split, (idx_train, idx_test) in enumerate(sss.split(X, Y)):
        rfs = RandomForestClassifier(n_estimators=10)
        Xtrain = X[idx_train] 
        Ytrain = Y[idx_train]
        Xtest = X[idx_test] 
        Ytest = Y[idx_test]
        siz = len(idx_train)
        rfs.fit(Xtrain, Ytrain)
        y_pred = rfs.predict(Xtest)
        acc.append(metrics.accuracy_score(Ytest, y_pred))
        print(metrics.accuracy_score(Ytest, y_pred))
    #accs.append(np.median(acc))
    #samples.append(siz)
    #print(samples,accs)

    importances = rfs.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rfs.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]
    names = []
    for idx in indices:
        names.append(featurenames[idx])
    print("Feature ranking:")
    
    with open('Ranking.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|',
                        quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        for f in range(X.shape[1]):
            print("%d. feature %s (%f)" % (f + 1, names[f], importances[indices[f]]))
            a = names[f]
            b = str(importances[indices[f]])
            spamwriter.writerow([a, b])


    #i_tree = 0
    #for tree_in_forest in rfs.estimators_:
    #    with open('tree_' + str(i_tree) + '.dot', 'w') as my_file:
    #        my_file = tree.export_graphviz(tree_in_forest, out_file = my_file)

    ## With XGboost

In [ ]:
def lightgbm():
    PARAMS_LGB = {
    "objective": "regression",
    "metric": "accuracy",
    "num_leaves": 30,
    "learning_rate": 0.1,
    "bagging_fraction": 0.7,
    "feature_fraction": 0.7,
    "bagging_freq": 5,
    "bagging_seed": 2018,
    "verbosity": -1}

    X = np.loadtxt("../../../MATLAB/XSHHS.csv", delimiter=",")
    Y = np.loadtxt("../../../MATLAB/YSHHS.csv", delimiter=",")
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2)
    train_data = lgb.Dataset(Xtrain, label=Ytrain)
    test_data = lgb.Dataset(Xtest, label=Ytest)
    num_round = 10
    bst = lgb.train(PARAMS_LGB, train_data, num_round, valid_sets=[test_data])
    train_data = lgb.Dataset(X, label=Y)
    bst = lgb.cv(param, train_data, num_round, nfold = 10)
 

In [ ]:
   
def feedforwardnetwork():
    clear = lambda: os.system('cls')
    clear() # remove warnings
    ################################ Load data ############################
    X = np.loadtxt("../../../MATLAB/XSHHS.csv", delimiter=",")
    Y = np.loadtxt("../../../MATLAB/YSHHS.csv", delimiter=",")
    ############################ Create model ##########################
    # baseline
    #neurons = [
    #    1024,
    #    256,
    #    256,
    #    256,
    #    256,
    #    512,
    #    256,
    #    512
    #           ]

    neurons = [
        2048,
        512,
        512,
        512,
               ]

    #neurons = [1024]

    actFuncs = [
        'relu',
        'relu',
        'relu',
        'relu',
                ]

    if len(actFuncs) != len(neurons):
        raise ValueError('No. hidden layers not consistent!')

    nHidLays = len(neurons)
    model = Sequential()
    ################################# Input layer
    ################################# #################################
    model.add(Dense(X.shape[1],input_dim = X.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    reg = 0.01

    ################################# Hidden layer(s)
    ################################# #################################
    for l in range(nHidLays):
        model.add(Dense(neurons[l], activation=actFuncs[l], kernel_regularizer=l2(reg)))

    ################################# Output layer
    ################################# #################################
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())

    
    mdlName = str.replace('./model_{}'.format(datetime.now().isoformat()[11:19]),':','') 
    ## Callbacks
    tensorboard = TensorBoard(log_dir="./Graph/{}".format(datetime.now().isoformat()[:4]), 
                              write_images=True)
    checkpoint = ModelCheckpoint(mdlName + '.h5',
                                monitor='val_acc',
                                verbose=0,
                                mode='max',
                                save_best_only=True)
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=20,
                              verbose=0, 
                              mode='auto')
    # Optimizers
    #opt = Adam(lr=0.01, decay=0.0)
    opt = SGD(lr=0.001, momentum=0.9, decay=0.0)
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])

    ## Save model to json
    #model_json = model.to_json()
    #with open(mdlName + ".json", "w") as json_file:
    #    json_file.write(model_json)

    # Split data into training and validation
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.4)

    # Split validation into validation and test
    Xval, Xtest, Yval, Ytest = train_test_split(Xtest, Ytest, test_size=0.5)

    ## Train
    model.fit(Xtrain, Ytrain, 
              validation_data=(Xval,Yval), 
              epochs=1000, 
              batch_size=16, 
              callbacks=[tensorboard,checkpoint,earlystop],
              verbose=2)
    
    # Save test,validation and predictions
    #testData = np.append(Xtest,Ytest,axis=1)
    #np.savetxt(mdlName + '_test.csv', testData)

    #valData = np.append(Xval,Yval,axis=1)
    #np.savetxt(mdlName + '_val.csv', valData)

    #DisplayBestModel(mdlName)

def DisplayBestModel(file):
    # load json and create model
    json_file = open(file + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(file + '.h5')
    opt = Adam(lr=0.001, decay=0.0)
    loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    data = np.loadtxt(file + '_val.csv')
    X = data[:,:-2]
    Y = data[:,-2:]
    score = loaded_model.evaluate(X, Y)
    print("%s: %.2f%%" % ('Validation accuracy', score[1] * 100))
    data = np.loadtxt(file + '_test.csv')
    X = data[:,:-2]
    Y = data[:,-2:]
    score = loaded_model.evaluate(X, Y)
    print("%s: %.2f%%" % ('Test accuracy', score[1] * 100))

def Metrics():
    #file = '../../4.6/model_115405_75'
    file = 'model_104420'
    # load json and create model
    json_file = open(file + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(file + '.h5')
    opt = Adam(lr=0.001, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    data = np.loadtxt(file + '_test.csv')
    Xval = data[:,:-2]
    Yval = data[:,-2:]
    valAcc = model.evaluate(Xval, Yval)
    pC1 = model.predict(Xval)[:,0]
    true1 = Yval[:,0]
    true0 = Yval[:,1]
    t = 0
    i = 0
    TPR = []
    FPR = []

    FPR, TPR, T = roc_curve(true1,pC1)
    plt.figure()
    plt.plot(FPR, TPR)
    plt.show()

    F1 = []
    for t in T:
        c1 = (pC1 >= t) * 1
        F1.append(f1_score(true1,c1))

    plt.figure()
    plt.plot(F1)
    plt.show()

In [ ]:
def BestXY():
    X = np.loadtxt("../../../MATLAB/X.csv", delimiter=",")
    Y = np.loadtxt("../../../MATLAB/Y.csv", delimiter=",")
    Xz = X #stats.zscore(X)
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    # convert integers to dummy variables (i.e.  one hot encoded)
    dummy_y = np_utils.to_categorical(encoded_Y)
    ############################ Create model ##########################
    # Fit the model
    # baseline
    reg = 0.01
    neurons = [1024,
               256,
               256,
               256,
               256,
               512,
               256,
               512]

    actFuncs = ['relu',
                'relu',
                'relu',
                'relu',
                'relu',
                'relu',
                'relu',
                'relu']

    if len(actFuncs) != len(neurons):
        raise ValueError('No. hidden layers not consistent!')

    nHidLays = len(neurons)
    model = Sequential()
    ################################# Input layer
    ################################# #################################
    model.add(Dense(X.shape[1],input_dim= X.shape[1], activation='linear'))
    #model.add(Dropout(0.5))

    ################################# Hidden layer(s)
    ################################# #################################
    for l in range(nHidLays):
        #model.add(BatchNormalization(axis=-1))
        model.add(Dense(neurons[l], activation=actFuncs[l], kernel_regularizer=l2(reg)))

    ################################# Output layer
    ################################# #################################
    model.add(Dense(dummy_y.shape[1], activation='softmax'))
    print(model.summary())

    #opt = Adam(lr=0.01, decay=0.1)
    opt = SGD(lr=0.01)
    
    mdlName = str.replace('./model_{}'.format(datetime.now().isoformat()[11:19]),':','') 
    ## Callbacks
    tensorboard = TensorBoard(log_dir="./Graph/{}".format(datetime.now().isoformat()[:4]), 
                              write_images=True)
    checkpoint = ModelCheckpoint(mdlName + '.h5',
                                monitor='val_acc',
                                verbose=0,
                                mode='max',
                                save_best_only=True)
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=20,
                              verbose=0, 
                              mode='auto')
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    # Save model to json
    model_json = model.to_json()
    with open(mdlName + ".json", "w") as json_file:
        json_file.write(model_json)

    # Split data into training and validation
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xz, dummy_y, test_size=0.4)

    # Split validation into validation and test
    Xval, Xtest, Yval, Ytest = train_test_split(Xtest, Ytest, test_size=0.5)

    ## Train
    model.fit(Xtrain, Ytrain, validation_data=(Xval,Yval), epochs=1000, batch_size=64, callbacks=[tensorboard,checkpoint])
    
    # Save test,validation and predictions
    testData = np.append(Xtest,Ytest,axis=1)
    np.savetxt(mdlName + '_test.csv', testData)

    valData = np.append(Xval,Yval,axis=1)
    np.savetxt(mdlName + '_val.csv', valData)

    DisplayBestModel(mdlName)


    